## Section 1: Import Libraries and load cifar10 dataset.

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from torch.utils.data import SubsetRandomSampler
#import additional libraries if needed

In [2]:
def load_cifar10_data(batch_size):
# load both training and test datasets, and transform them to tensors.
  transform = transforms.Compose(
      [transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

  trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                          download=True, transform=transform)
  trainset.data = trainset.data[:30000]
  trainset.targets = trainset.targets[:30000]
  print("train",len(trainset))
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True, num_workers=2)

  testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)
  #testset.data = testset.data[:2000]
  #testset.targets = testset.targets[:2000]
  print("test",len(testset))
  testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=False, num_workers=2)
  
  return trainset, testset, trainloader, testloader

In [3]:
batch_size = 4
trainset, testset, trainloader, testloader = load_cifar10_data(batch_size)
classes = ('plane', 'car', 'bird', 'cat',
            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## Check the dimensions of a batch:
for images, labels in trainloader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Files already downloaded and verified
train 30000
Files already downloaded and verified
test 10000
Image batch dimensions: torch.Size([4, 3, 32, 32])
Image label dimensions: torch.Size([4])


## Section 2: Load Pre-trained Model 

## Section 2.1: ALEXNET

In [4]:
from torchvision import models

# Load the pre-trained ResNet50 model
model = models.resnet50(pretrained=True)



# Print modified RESNet model

print(model)

C:\Users\yasmi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\yasmi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
import torch.nn as nn

for param in model.parameters():
    param.requires_grad = False



# Replace the final layer to output 10 classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)






## Section 3: Define optimizer and loss criterion

In [6]:
# Hyperparameters
random_seed = 42
learning_rate = 0.001
num_epochs = 10

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

# loss
loss = nn.CrossEntropyLoss()

In [7]:
"""DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', DEVICE)
# Set my pretrained model to be run on GPU
model = model.to(DEVICE)"""

'DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")\nprint(\'Device:\', DEVICE)\n# Set my pretrained model to be run on GPU\nmodel = model.to(DEVICE)'

## Section 4: Train the model

In [8]:
def train_model(trainset, num_epochs, model, optimizer, loss):
    # split data into 10 folds
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    fold_losses = []
    fold_val_accs = []

    for fold, (train_idx, val_idx) in enumerate(kfold.split(trainset)):
        print(f"Fold {fold + 1}")

        # get fold data
        train_fold = torch.utils.data.Subset(trainset, train_idx)
        val_fold = torch.utils.data.Subset(trainset, val_idx)

        # create data loaders
        batch_size = 32
        trainloader = torch.utils.data.DataLoader(train_fold, batch_size=batch_size,
                                                  shuffle=True, num_workers=2)
        valloader = torch.utils.data.DataLoader(val_fold, batch_size=batch_size,
                                                shuffle=False, num_workers=2)

        # train for num_epochs epochs
        for epoch in range(num_epochs):
            running_loss = 0.0

            for i, (inputs, labels) in enumerate(trainloader):
                optimizer.zero_grad()

                outputs = model(inputs)
                loss_value = loss(outputs, labels)
                loss_value.backward()
                optimizer.step()

                running_loss += loss_value.item()

            # calculate validation accuracy
            correct = 0
            total = 0
            with torch.no_grad():
                for val_inputs, val_labels in valloader:
                    val_outputs = model(val_inputs)
                    _, predicted = torch.max(val_outputs.data, 1)
                    total += val_labels.size(0)
                    correct += (predicted == val_labels).sum().item()

            val_acc = correct / total
            fold_val_accs.append(val_acc)

            print(f"Epoch {epoch + 1}: Loss={running_loss:.4f}, Validation Accuracy={val_acc:.4f}")

        fold_losses.append(running_loss)

    # calculate final validation accuracy
    mean_val_acc = sum(fold_val_accs) / len(fold_val_accs)
    print(f"Final Validation Accuracy: {mean_val_acc:.4f}")

In [9]:
train_model(trainset, num_epochs, model, optimizer, loss)

Fold 1
Epoch 1: Loss=1485.5171, Validation Accuracy=0.4383
Epoch 2: Loss=1325.5450, Validation Accuracy=0.4680
Epoch 3: Loss=1282.8775, Validation Accuracy=0.4700
Epoch 4: Loss=1257.5849, Validation Accuracy=0.4803
Epoch 5: Loss=1236.4369, Validation Accuracy=0.4813
Epoch 6: Loss=1222.0287, Validation Accuracy=0.4933
Epoch 7: Loss=1211.8514, Validation Accuracy=0.4987
Epoch 8: Loss=1194.5022, Validation Accuracy=0.4970
Epoch 9: Loss=1199.2176, Validation Accuracy=0.4923
Epoch 10: Loss=1185.0034, Validation Accuracy=0.4957
Fold 2
Epoch 1: Loss=1183.6852, Validation Accuracy=0.5133
Epoch 2: Loss=1180.1260, Validation Accuracy=0.5187
Epoch 3: Loss=1178.4146, Validation Accuracy=0.5140
Epoch 4: Loss=1165.7075, Validation Accuracy=0.5257
Epoch 5: Loss=1174.4910, Validation Accuracy=0.5240
Epoch 6: Loss=1159.8247, Validation Accuracy=0.5173
Epoch 7: Loss=1161.2848, Validation Accuracy=0.5120
Epoch 8: Loss=1160.0832, Validation Accuracy=0.5280
Epoch 9: Loss=1160.6201, Validation Accuracy=0.51

## Section 5: Test the model

Write a function to test the model using testloader.

In [10]:
dataiter = iter(testloader)
images, labels = next(dataiter)

In [11]:
#Test for the entire dataset

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 33 %
